In [1]:
import requests
import json
import pandas as pd

In [2]:
with open('keyfile') as keyfile:
    API_KEY= keyfile.read()
    
## Geography
london_box = (51.145756,-0.913796,51.955268,0.388083)
london_mid = (51.5,-0.12)
# Trafalgar Square
uk_box = (49.3739906, -7.0113373, 61.2291139, 2.4798203)
# Isles of Scilly to Shetland Islands (includes a bit of belgium and most of ROI)

# Requests
url = 'https://airlabs.co/api/v9/'
endpoint = lambda e: url + e

# Get Static Data
with open('airports.json') as airport_data:
    airports = json.load(airport_data)
    
with open('ldn_airports.json') as ldn_airports_file:
    ldn_airports = json.load(ldn_airports_file)
    
ldn_names = [a['name'] for a in ldn_airports]    

# Dictionary for turning iata_codes into airport names
#TO DO: add cities to this

codes = {ap['iata_code']:ap['name'] 
         for ap in airports if 'iata_code' in ap.keys()
        }

In [3]:
# Request live data

r= requests.get(endpoint('flights'),
                params={
                    'api_key':API_KEY,
                    'bbox':london_box,
                }
               )
all_planes = r.json()['response']
flying_planes = [plane for plane in all_planes if plane['status'] == 'en-route']

# make DataFrame

df= pd.DataFrame(flying_planes)
#print(df.columns); df.head()

# Make the dataframe more user friendly

df['Source'] = df['dep_iata'].map(lambda x : codes[x], 
                                  na_action='ignore')
df['Destination'] = df['arr_iata'].map(lambda x : codes[x], 
                                       na_action='ignore')

# Divide frames into arrivals, departures, fly-overs, mystery and intra-london (naughty) flights
df_to_london = df.query("Destination in @ldn_names")
df_from_london = df.query("Source in @ldn_names")
df_flyover = df.query("Source not in @ldn_names and Destination not in @ldn_names").dropna()
df_mystery = df.query('Destination.isna()')
df_naughty = df_to_london.query('Source in @ldn_names')

# cheeky Lambda for representing the dataframes
show_frame = lambda df : df[['Source','Destination','alt','reg_number','flag']].sort_values(['alt','Destination'])

### If there are any planes flying between london airports AND they've put this in their trackers, they will show up here:

In [4]:
if len(df_naughty)>0:
    show_frame(df_naughty)
else:
    print("no planes")

no planes


In [5]:
show_frame(df_flyover)

,Source,Destination,alt,reg_number,flag
8,Mildenhall Royal Air Force Base,Dover Air Force Base,10972,N487MC,US
1,Gran Canaria Airport,East Midlands Airport,10972,G-JZHD,UK
4,Paris Charles de Gaulle Airport,Dublin Airport,10980,EI-DEJ,IE
7,Osmani International Airport,Manchester Airport,11582,S2-AJU,BD
0,Munich International Airport,Chicago O'Hare International Airport,11590,D-AIXL,DE
2,Malaga Airport,"Oslo Airport, Gardermoen",12192,LN-FGJ,NO


In [6]:
show_frame(df_from_london)

,Source,Destination,alt,reg_number,flag
50,London Heathrow Airport,Los Angeles International Airport,-121,G-VRNB,UK
64,London Gatwick Airport,Istanbul Airport,-68,TC-LSK,TR
21,London Heathrow Airport,Aberdeen International Airport,0,G-TTNF,UK
31,London Heathrow Airport,Billund Airport,0,G-MIDO,UK
32,London Gatwick Airport,Edinburgh Airport,0,G-EZBE,UK
76,London Heathrow Airport,John F. Kennedy International Airport,0,N728AN,US
75,London Heathrow Airport,Logan International Airport,0,N661UA,US
30,London Heathrow Airport,Newark Liberty International Airport,0,G-YMMO,UK
38,London Gatwick Airport,Zurich Airport,0,G-EZGN,GB
52,London Heathrow Airport,Chhatrapati Shivaji International Airport,472,G-XWBM,UK


In [7]:
show_frame(df_to_london)

,Source,Destination,alt,reg_number,flag
67,Cork Airport,London Heathrow Airport,-99,EI-DVL,IE
23,Antalya Airport,London Gatwick Airport,0,G-UZLC,GB
37,Newcastle International Airport,London Heathrow Airport,0,G-ECOR,GB
71,Stockholm Arlanda Airport,London Heathrow Airport,0,EI-SIH,SE
59,Henri Coanda International Airport,London Luton Airport,0,HA-LZR,HU
77,Indianapolis International Airport,London Stansted Airport,0,N849FD,US
61,Faro Airport,London Biggin Hill Airport,68,OK-KIN,CZ
62,Zurich Airport,London City Airport,457,HB-AZA,CH
33,Faro Airport,London Luton Airport,899,G-EZBH,GB
69,Copenhagen Airport,London Heathrow Airport,960,EI-SIN,SE
